In [ ]:
#| default_exp main
import sys,os
from pathlib import Path
# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))
%load_ext autoreload
%autoreload 2

# Principal
> Este módulo concentra funções auxiliares específicas que filtram os dados do banco com campos e formatação de interesse para aplicações específicas como o [appAnalise](https://github.com/EricMagalhaesDelgado/appAnalise) por exemplo.

In [ ]:
#| export
import json
from datetime import datetime
from pathlib import Path
from typing import Union

import pandas as pd
import pyodbc
from dotenv import find_dotenv, load_dotenv
from fastcore.test import *
from pymongo import MongoClient
from rich import print

from extracao.format import _filter_matlab, _format_matlab, merge_on_frequency
from extracao.reading import read_aero, read_base

load_dotenv(find_dotenv(), override=True)

/home/melinda/micromamba/envs/dados/lib/python3.8/site-packages/fastprogress/fastprogress.py:107: UserWarning: Couldn't import ipywidgets properly, progress bar will use console behavior
  warn("Couldn't import ipywidgets properly, progress bar will use console behavior")


ModuleNotFoundError: No module named 'extracao.aero'

In [ ]:
#| export
def get_db(
    path: Union[str, Path],  # Pasta onde salvar os arquivos",
    connSQL: pyodbc.Connection = None,  # Objeto de conexão do banco SQL Server
    clientMongoDB: MongoClient = None,  # Objeto de conexão do banco MongoDB
) -> pd.DataFrame:  # Retorna o DataFrame com as bases da Anatel e da Aeronáutica
    """Lê e opcionalmente atualiza as bases da Anatel, mescla as bases da Aeronáutica, salva e retorna o arquivo
    A atualização junto às bases de dados da Anatel é efetuada caso ambos objetos de banco `connSQL` e `clientMongoDB` forem válidos`
    """
    dest = Path(path)
    dest.mkdir(parents=True, exist_ok=True)
    print(":scroll:[green]Lendo as bases de dados da Anatel...")
    df = read_base(path, connSQL, clientMongoDB)
    df = _filter_matlab(df)
    mod_times = {"ANATEL": datetime.now().strftime("%d/%m/%Y %H:%M:%S")}
    print(":airplane:[blue]Requisitando os dados da Aeronáutica.")
    update = all([connSQL, clientMongoDB])
    try:
        aero = read_aero(path, update=update)
    except Exception:
        aero = read_aero(path)
    mod_times["AERONAUTICA"] = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    print(":spoon:[yellow]Mesclando os dados da Aeronáutica.")
    df = _format_matlab(merge_on_frequency(df, aero))
    print(":card_file_box:[green]Salvando os arquivos...")
    df.to_parquet(f"{dest}/AnatelDB.parquet.gzip", compression="gzip", index=False)
    versiondb = json.loads((dest / "VersionFile.json").read_text())
    mod_times["ReleaseDate"] = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    versiondb["anateldb"].update(mod_times)
    json.dump(versiondb, (dest / "VersionFile.json").open("w"))
    print("Sucesso :zap:")
    return df

In [ ]:
#|eval: false
import os
import warnings
from extracao.updates import connect_db
import numpy as np
from geopy.distance import geodesic


In [ ]:
#|eval: false
warnings.filterwarnings("ignore")

In [ ]:
#|eval: false
folder = Path.cwd().parent / 'dados'

In [ ]:
#|eval: false
# conn = connect_db()
# uri = os.environ['MONGO_URI']
# mongo_client = MongoClient(uri)

In [ ]:
#|eval: false
# os.chdir(f"{Path.cwd().parent}/extracao/aero")
df = get_db(folder)
# df = get_db(folder) #, conn, mongo_client)
# df = pd.read_parquet(folder / 'AnatelDB.parquet.gzip')

📜Lendo as bases de dados da Anatel...

✈Requisitando os dados da Aeronáutica.

🥄Mesclando os dados da Aeronáutica.

🗃Salvando os arquivos...

Sucesso ⚡

In [ ]:
#|eval: false
from pandas_profiling import ProfileReport
df['Frequency'] = df['Frequency'].astype('category')
profile = ProfileReport(df, config_file= folder.parent / 'nbs/report_config.yaml')
profile.to_notebook_iframe()